In [1]:
## imports 
import pandas as pd
import numpy as np
import yaml
import plotnine
from plotnine import *

## way to connect to mysql 
import mysql.connector

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)

pd.options.display.max_rows = 999
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preliminary: define connection and read sample of data

In [2]:
## read in creds; change the path name if stored
## elsewhere
creds = load_creds("../../../private_data/qss20_w22_dbcred.yaml")


In [3]:
## connect to the database
cnx = mysql.connector.connect(user=creds['practice_database']['user'], 
                            password=creds['practice_database']['password'],
                            port=creds['practice_database']['port'],
                            database= creds['practice_database']['database'],
                            host = creds['practice_database']['host'])
cnx

# Activity 1

1. Create a new column -- `in_chicago` when pulling from the `caseinit` table that takes on the value of "YES" if INCIDENT_CITY = Chicago; "NO" otherwise (which represents incidents in Cook County suburbs outside the city limits);  and pull the table. Use `crosstabs` to confirm that this worked
2. Repeat step 1 but also filter out blank strings (`INCIDENT_CITY` == "")
3. Use `where` to row filter to initiations in Chicago and use group by to find the count of cases diverted and not diverted (`is_in_diversion`); pull the table with those counts
4. Modify the query in step 3 to find the proportion of cases in chicago diverted (hint you made need to use case when in a subquery)
5. Modify the query in step 4 to find the proportion of cases in chicago versus cases not in chicago sent to diversion 


In [31]:
## step 1
query_createvar = """
select *,
CASE when INCIDENT_CITY = 'Chicago' then 'Yes'
ELSE 'No'
END as in_chicago
from caseinit
"""

varcreate = pd.read_sql_query(query_createvar, cnx)
pd.crosstab(varcreate.in_chicago, varcreate.INCIDENT_CITY).T.head()
pd.crosstab(varcreate.in_chicago, varcreate.INCIDENT_CITY).T.tail()

## step 2
query_createvar_noblank = """
select *,
CASE when INCIDENT_CITY = 'Chicago' then 'Yes'
ELSE 'No'
END as in_chicago
from caseinit
where INCIDENT_CITY <> ""
"""

varcreate_nb = pd.read_sql_query(query_createvar_noblank, cnx)
pd.crosstab(varcreate_nb.in_chicago, varcreate_nb.INCIDENT_CITY).T.head(20)
pd.crosstab(varcreate_nb.in_chicago, varcreate_nb.INCIDENT_CITY).T.tail(15)


in_chicago,No,Yes
INCIDENT_CITY,,
,178,0
Alsip,21,0
Arlington Heights,20,0
Bartlett,8,0
Bedford Park,7,0


in_chicago,No,Yes
INCIDENT_CITY,,
Western Springs,1,0
Wheeling,17,0
Wilmette,2,0
Winnetka,4,0
Worth,5,0


in_chicago,No,Yes
INCIDENT_CITY,,
Alsip,21,0
Arlington Heights,20,0
Bartlett,8,0
Bedford Park,7,0
Bellwood,8,0
Berkeley,5,0
Berwyn,25,0
Blue Island,21,0
Bridgeview,18,0


in_chicago,No,Yes
INCIDENT_CITY,,
South Holland,19,0
Steger,2,0
Stickney,1,0
Stone Park,2,0
Streamwood,19,0
Summit,15,0
Thornton,3,0
Tinley Park,20,0
Waukegan,2,0


In [32]:
## step 3
query_countdiv = """
select count(*) as count_div, is_in_diversion
from caseinit
where INCIDENT_CITY = 'Chicago'
group by is_in_diversion
"""

In [33]:
count_div = pd.read_sql_query(query_countdiv, cnx)
count_div

## step 4
query_propdiv = """
select avg(is_div_bool) as prop_div
from  (select *,
CASE when is_in_diversion = 'True' then 1
ELSE 0
END is_div_bool
from caseinit) as d
where INCIDENT_CITY = 'Chicago'
"""
prop_div = pd.read_sql_query(query_propdiv, cnx)
prop_div

## step 5
query_propdiv_compare = """
select avg(is_div_bool) as prop_div, in_chicago
from  (select *,
CASE when is_in_diversion = 'True' then 1
ELSE 0
END as is_div_bool,
CASE when INCIDENT_CITY = 'Chicago' then 'Yes'
ELSE 'No'
END as in_chicago
from caseinit) as d
where INCIDENT_CITY <> ''
group by in_chicago
"""
prop_div_compare = pd.read_sql_query(query_propdiv_compare, cnx)
prop_div_compare


,count_div,is_in_diversion
0,1702,False
1,1394,True


,prop_div
0,0.4503


,prop_div,in_chicago
0,0.5956,No
1,0.4503,Yes


# Activity 2 

1. Use the following crosswalk and the `CASE` variable in the `divert` table to create a new variable `DIVERSION_PROGRAM_TEXT` that spells out the diversion programs

DC: Drug Court

DDPP: Drug Deferred Prosecution

DS: Drug School

RJCC: Restorative Justice

MHC: Mental Health Court

VC: Veteran Court

2. Build on the query from step 1 to filter to Narcotics as the `UPDATED_OFFENSE_CATEGORY` and Black or White defendants (based on race in the diversions table) (hint: you'll need to join with the caseinit table based on case_id and case_participant_id, you can do a inner join to keep only those diverted). Select the case_id, case_participant_id, case, race, and diversion_program_text columns

3. Built on the query from step 2 (and/or modify to just focus on drug school and drug court) to find the (1) rate of Black defendants sent to drug court, (2) rate of white defendants sent to drug court, (3) rate of Black defendants sent to drug school, and (4) rate of white defendants sent to drug school


In [4]:
## step 1; query creating var
dp_type = """
select *,
    CASE WHEN DIVERSION_PROGRAM = 'DC' THEN 'Drug Court'
    WHEN DIVERSION_PROGRAM = 'DDPP' THEN 'Drug Deferred Prosecution'
    WHEN DIVERSION_PROGRAM = 'DS' THEN 'Drug School'
    WHEN DIVERSION_PROGRAM = 'RJCC' THEN 'Restorative Justice'
    WHEN DIVERSION_PROGRAM = 'MHC' THEN 'Mental Health Court'
    WHEN DIVERSION_PROGRAM = 'VC' THEN 'Veteran Court'
ELSE 'Other'
END as DIVERSION_PROGRAM_TEXT
from divert
"""

In [5]:
n = pd.read_sql_query(dp_type, cnx)
pd.crosstab(n.DIVERSION_PROGRAM,
           n.DIVERSION_PROGRAM_TEXT)

DIVERSION_PROGRAM_TEXT,Drug Court,Drug Deferred Prosecution,Drug School,Mental Health Court,Other,Restorative Justice,Veteran Court
DIVERSION_PROGRAM,,,,,,,
ACT,0,0,0,0,64,0,0
ARI,0,0,0,0,148,0,0
BR9,0,0,0,0,561,0,0
DC,443,0,0,0,0,0,0
DDPP,0,471,0,0,0,0,0
DS,0,0,558,0,0,0,0
MHC,0,0,0,278,0,0,0
RJCC,0,0,0,0,0,30,0
VC,0,0,0,0,0,0,117


In [6]:
## step 2: add the filters 
dp_type_filtered = """
select c.CASE_ID, c.CASE_PARTICIPANT_ID,
d.RACE, DIVERSION_PROGRAM_TEXT
from  (select *,
CASE WHEN DIVERSION_PROGRAM = 'DC' THEN 'Drug Court'
    WHEN DIVERSION_PROGRAM = 'DDPP' THEN 'Drug Deferred Prosecution'
    WHEN DIVERSION_PROGRAM = 'DS' THEN 'Drug School'
    WHEN DIVERSION_PROGRAM = 'RJCC' THEN 'Restorative Justice'
    WHEN DIVERSION_PROGRAM = 'MHC' THEN 'Mental Health Court'
    WHEN DIVERSION_PROGRAM = 'VC' THEN 'Veteran Court'
ELSE 'Other'
END as DIVERSION_PROGRAM_TEXT
from divert) as d
inner join (
    select UPDATED_OFFENSE_CATEGORY,
    CASE_ID, CASE_PARTICIPANT_ID
    from caseinit
    ) as c
on c.CASE_ID = d.CASE_ID and
c.CASE_PARTICIPANT_ID = d.CASE_PARTICIPANT_ID
where d.RACE IN ("Black", "White")
and UPDATED_OFFENSE_CATEGORY = 'Narcotics'
"""

In [7]:
d_filtered = pd.read_sql_query(dp_type_filtered, cnx)
d_filtered.head()


,CASE_ID,CASE_PARTICIPANT_ID,RACE,DIVERSION_PROGRAM_TEXT
0,432664409023,863972841464,Black,Drug Court
1,432664409023,863972841464,Black,Drug Deferred Prosecution
2,440622023126,888635038783,Black,Drug Deferred Prosecution
3,414180986315,804998415688,White,Drug School
4,398778721478,756959653773,White,Drug School


In [8]:
## step 3: add grouping and 1,0 calc
dp_compare = """
select d.race, avg(is_dc) as prop_drugcourt, avg(is_ds) as prop_drugschool
from  (select *,
CASE WHEN DIVERSION_PROGRAM = 'DS' THEN 1 ELSE 0 END as is_ds,
CASE WHEN DIVERSION_PROGRAM = 'DC' THEN 1 ELSE 0 END as is_dc
from divert
) as d
inner join (
    select UPDATED_OFFENSE_CATEGORY,
    CASE_ID, CASE_PARTICIPANT_ID
    from caseinit
    ) as c
on c.CASE_ID = d.CASE_ID and
c.CASE_PARTICIPANT_ID = d.CASE_PARTICIPANT_ID
where d.RACE IN ("Black", "White")
and UPDATED_OFFENSE_CATEGORY = 'Narcotics'
group by d.race
"""

In [10]:
pd.read_sql_query(dp_compare, cnx)

## white defendants sent to drug school at much higher rates
## proportions dont sum to 1 due to other programs 

,RACE,prop_drugcourt,prop_drugschool
0,Black,0.1620,0.1877
1,White,0.1273,0.5324
